### Import Stuff

In [ ]:
!pip install radis
!pip install memory_profiler

from radis.db.classes import get_molecule_identifier
from radis.levels.partfunc import PartFuncHAPI

from radis.io.hitemp import fetch_hitemp
from radis.db.classes import get_molecule
from radis.phys.constants import hc_k

import numpy as np
from numpy import exp, pi

%load_ext memory_profiler

### Function to retrieve Qgas value from partition table

In [62]:
def get_Qgas(molecule, iso, T):

    M = get_molecule_identifier(molecule)

    Q = PartFuncHAPI(M, iso)
    return Q.at(T=T)

### Convert list to dictionary 

In [63]:
def list_to_dict(list0):

  dict0 = dict() 
  for index,value in enumerate(list0):
    dict0[index] = value
  
  return dict0

### Old function to used to scale linestrength values 

In [64]:
def calc_linestrength_eq(df, Tref, Tgas):
  if len(df) == 0:
          return

    
  print("Scaling equilibrium linestrength")

  # %% Load partition function values

  def _calc_Q(molecule, iso, T_ref, T_gas):

      Qref = get_Qgas(molecule, iso, T_ref)
      Qgas = get_Qgas(molecule, iso, T_gas)


      return Qref, Qgas

  id_set = df.id.unique()
  id = list(id_set)[0]
  molecule = get_molecule(id) # retrieve the molecule
  iso_set = set(df.iso)  # df1.iso.unique()

  # when we have only one isotope Qref, Qgas are stored as attributes
  if len(iso_set) == 1:
      Qref, Qgas = _calc_Q(molecule, iso_set[0], T_gas, T_ref)
      df.Qref = float(Qref) 
      df.Qgas = float(Qgas)
      assert "Qref" not in df.columns
      assert "Qgas" not in df.columns

  # when we have more than one isotope Qref, Qgas are stored as separate columns
  else:

      iso_arr = list(range(max(iso_set) + 1))

      Qref_arr = np.empty_like(iso_arr, dtype=np.float64)
      Qgas_arr = np.empty_like(iso_arr, dtype=np.float64)
      for iso in iso_arr:
          if iso in iso_set:
              Qref, Qgas = _calc_Q(molecule, iso, Tref, Tgas)
              Qref_arr[iso] = Qref
              Qgas_arr[iso] = Qgas

      df["Qref"] = Qref_arr.take(df.iso)
      df["Qgas"] = Qgas_arr.take(df.iso)

  # Scaling linestrength with the equations from Rotham's paper
  line_strength = df.int * (df.Qref / df.Qgas)
  #line_strength = df.int * (Qref_arr.take(df.iso) / Qgas_arr.take(df.iso))
  line_strength *= exp(-hc_k * df.El * (1 / Tgas - 1 / Tref))
  line_strength *= (1 - exp(-hc_k * df.wav / Tgas)) / (
      1 - exp(-hc_k * df.wav / Tref)
  )
  # Add a fresh columns with the scaled linestrength
  df["S"] = line_strength  # [cm-1/(molecules/cm-2)]

  # Just to make sure linestrength is indeed added
  assert "S" in df

  return df

### Scale linstrengths with the new method

In [85]:
def calc_linestrength_eq_optimised(df, Tref, Tgas):
  if len(df) == 0:
          return

  if "Qref" in df:
    print("yes Qref here")
  
  else:
    print("not Qref here")

    
  print("Scaling equilibrium linestrength")

  # %% Load partition function values

  def _calc_Q(molecule, iso, T_ref, T_gas):

      Qref = get_Qgas(molecule, iso, T_ref)
      Qgas = get_Qgas(molecule, iso, T_gas)


      return Qref, Qgas

  id_set = df.id.unique()
  id = list(id_set)[0]
  molecule = get_molecule(id) # retrieve the molecule
  iso_set = set(df.iso)  # df1.iso.unique()

  # when we have only one isotope Qref, Qgas are stored as attributes
  if len(iso_set) == 1:
      Qref, Qgas = _calc_Q(molecule, iso_set[0], T_gas, T_ref)
      df.Qref = float(Qref) 
      df.Qgas = float(Qgas)
      assert "Qref" not in df.columns
      assert "Qgas" not in df.columns

  # when we have more than one isotope Qref, Qgas are stored as separate columns
  else:

      iso_arr = list(range(max(iso_set) + 1))

      Qref_arr = np.empty_like(iso_arr, dtype=np.float64)
      Qgas_arr = np.empty_like(iso_arr, dtype=np.float64)
      for iso in iso_arr:
          if iso in iso_set:
              Qref, Qgas = _calc_Q(molecule, iso, Tref, Tgas)
              Qref_arr[iso] = Qref
              Qgas_arr[iso] = Qgas
      
      # convert dictionaries to lists
      Qref_dict = list_to_dict(Qref_arr)
      Qgas_dict = list_to_dict(Qgas_arr)

      ratio = {k: Qref_dict.get(k, 0) / Qgas_dict.get(k, 0) for k in set(Qref_dict) | set(Qgas_dict)}

      #copy iso column
      df["S"] = df["iso"]

      #map the ratios in the dictionary to the iso column vallues 
      df["S"].map(ratio).fillna(df["S"])

  # Scaling linestrength with the equations from Rotham's paper
  line_strength = df.int * (df.S)
  line_strength *= exp(-hc_k * df.El * (1 / Tgas - 1 / Tref))
  line_strength *= (1 - exp(-hc_k * df.wav / Tgas)) / (
      1 - exp(-hc_k * df.wav / Tref)
  )
  # Add a fresh columns with the scaled linestrength
  df["S"] = line_strength  # [cm-1/(molecules/cm-2)]

  # Just to make sure linestrength is indeed added
  assert "S" in df

  return df

### Fetch Hitemp

I am using HITEMP-CH4 to test this. 

In [89]:
Tref = 296 # The reference temperature is constant throughout radis

df0 = fetch_hitemp(molecule='CH4', databank_name='HITEMP-CH4', isotope='1, 2, 3', load_wavenum_min=2000, load_wavenum_max=3000)

print("================Before computation=====================")
print(df0.head())

Tgas = 300 # Any temperature of your choice at which you would like to scale linestrength

%time df0 = calc_linestrength_eq(df0, Tref, Tgas)

print("================After computation Old=====================")
print(df0.head())
df0.info(verbose=False, memory_usage="deep")

Using existing database HITEMP-CH4
================Before computation=====================
       id  iso         wav           int  ...          iref  lmix     gp    gpp
87326   6    1  2000.00250  2.852000e-42  ...  715947141313          0.0    0.0
87327   6    1  2000.00250  8.171000e-36  ...  715947141313          0.0    0.0
87328   6    1  2000.00264  3.091000e-37  ...  705832 3 3 3        255.0  245.0
87329   6    1  2000.00362  1.880000e-31  ...  705832 3 3 3        123.0  117.0
87330   6    1  2000.00375  1.288000e-39  ...  705832 3 3 3         90.0   86.0

[5 rows x 19 columns]
Scaling equilibrium linestrength
CPU times: user 1.02 s, sys: 29.7 ms, total: 1.05 s
Wall time: 982 ms
================After computation Old=====================
       id  iso         wav  ...        Qref      Qgas             S
87326   6    1  2000.00250  ...  590.477738  602.8155  5.820099e-42
87327   6    1  2000.00250  ...  590.477738  602.8155  1.292129e-35
87328   6    1  2000.00264  ...  590.477

In [90]:
df1 = fetch_hitemp(molecule='CH4', databank_name='HITEMP-CH4', isotope='1, 2, 3', load_wavenum_min=2000, load_wavenum_max=3000)

%time df1 = calc_linestrength_eq_optimised(df1, Tref, Tgas)

print("================After computation Optimised (?)=====================")
print(df1.head())
df1.info(verbose=False, memory_usage="deep")

Using existing database HITEMP-CH4
not Qref here
Scaling equilibrium linestrength


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in double_scalars


not Qref here
CPU times: user 425 ms, sys: 3.05 ms, total: 428 ms
Wall time: 385 ms
================After computation Optimised (?)=====================
       id  iso         wav           int  ...  lmix     gp    gpp             S
87326   6    1  2000.00250  2.852000e-42  ...          0.0    0.0  5.941707e-42
87327   6    1  2000.00250  8.171000e-36  ...          0.0    0.0  1.319127e-35
87328   6    1  2000.00264  3.091000e-37  ...        255.0  245.0  5.060155e-37
87329   6    1  2000.00362  1.880000e-31  ...        123.0  117.0  2.495688e-31
87330   6    1  2000.00375  1.288000e-39  ...         90.0   86.0  2.308681e-39

[5 rows x 20 columns]
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2042974 entries, 87326 to 30299
Columns: 20 entries, id to S
dtypes: float64(11), int64(2), object(7)
memory usage: 1.1 GB
